# Парсер

Сначала строим абстрактное синтаксическое дерево:

In [1]:
import re

def get_flat_list(code):
    lparen = r'\('
    rparen = r'\)'
    number = r'\d+(?:\.\d+)?'
    arith = r'\/\/|=|\*|\/|\+|\-'
    ident = r'[a-zA-Z_]+\d*'    
    tokens = '|'.join([lparen, rparen, number, arith, ident])
    return re.findall(tokens, code)

def parse_item(item):
    if re.match('^\d+\.\d+$', item):
        return float(item)
    elif re.match('^\d+$', item):
        return int(item)
    else:
        return item

def get_hier_list(flat_list, start=0):
    result = []
    i = start + 1
    while (flat_list[i] != ')'):
        if (flat_list[i] == '('):
            lst, i = get_hier_list(flat_list, i)
            result += [lst]
        else:
            result += [parse_item(flat_list[i])]
            i += 1
    return result, i + 1

def parse(code):
    return get_hier_list(get_flat_list(code))[0]

In [2]:
parse('(gcd 18 45)')

['gcd', 18, 45]

In [3]:
parse('(define double (lambda (n)(* n 2)))')

['define', 'double', ['lambda', ['n'], ['*', 'n', 2]]]

In [4]:
parse('(+ x 13)')

['+', 'x', 13]

In [5]:
parse('(gcd x 13)')

['gcd', 'x', 13]

In [6]:
parse('(define x 13)')

['define', 'x', 13]

In [7]:
parse('''(define (mod m n)
    (- m (* n (// m n))))''')

['define', ['mod', 'm', 'n'], ['-', 'm', ['*', 'n', ['//', 'm', 'n']]]]

In [8]:
parse('''(define (gcd m n)
    (if (= n 0)
        m
        (gcd n (mod m n))))''')

['define',
 ['gcd', 'm', 'n'],
 ['if', ['=', 'n', 0], 'm', ['gcd', 'n', ['mod', 'm', 'n']]]]